# IMPORT LIBRARY

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import requests
import time
import re

# START SCRAPING

In [208]:
name = []
price = []
province = []
sold = []

In [209]:
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
driver = webdriver.Chrome(executable_path=r'C:\Users\tanna\OneDrive\เดสก์ท็อป\DSI2022\chromedriver.exe', options=options)



for i in range(0,10):
    driver.get(f'https://shopee.co.th/search?keyword=อาหารออร์แกนิค&page={i}')
    time.sleep(3)
    if i == 0:
        thai_button = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
        thai_button.click()
        time.sleep(2)

    driver.execute_script("document.body.style.zoom='10%'")
    time.sleep(3)
    data = driver.page_source
    soup = bs(data)
    
    
    all_item = soup.find_all("div",{'class':'col-xs-2-4 shopee-search-item-result__item'})
    
    for i in all_item:
        a = bs(str(i))
        
        #ชื่อ
        n = a.find('div',{'class':'ie3A+n bM+7UW Cve6sh'}).get_text() #ดึงชื่อจาก tag
        name.append(n)
        
        #ราคา
        p = a.find('span',{'class':'ZEgDH9'}).get_text() #ดึงราคา
        price.append(p)
        
        #จังหวัด
        pro = a.find('div',{'class':'zGGwiV'}).get_text()
        pro = re.sub("จังหวัด","",pro)
        province.append(pro)
        
        #ขายแล้ว
        s = a.find('div',{'class':'ZnrnMl'}).get_text()
        if s == "":
            s = "-1"
        sold.append(s)
    


# DATA CLEANSING

In [217]:
#ปรับจำนวนการขายเป็นตัวเลข
amountsold = []
for i in sold:
    i = i.split()
    if len(i) == 1:
        amountsold.append(i[0])
    else:
        j = re.sub("พัน","k",i[1])
        if "k" in j:
            j = re.sub("k","",j)
            j = float(j)
            j *= 1000
            amountsold.append(j)
        else:
            j = float(j)
            amountsold.append(j)
        

#ปรับราคาเป็น float
price_num = []
for i in price:
    i = re.sub(",","",i)
    i = float(i)
    price_num.append(i)
    


#ปรับชื่อ
name_clean = []
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
for i in name:
    i = emoji_pattern.sub(r'', i)
    name_clean.append(i)

# FINAL

In [218]:
final = pd.DataFrame({'Name': name_clean,
                     'Price' : price_num,
                     'Province': province,
                     'AmountSold': amountsold})

In [219]:
final

,Name,Price,Province,AmountSold
0,Jinny (ส่งฟรี) ซอสเด็ก ผงปรุงรสเด็ก เซ็ตจับคู่...,420.0,อุทัยธานี,6
1,คิดส์พาสต้า5คุณประโยชน์ ผสมผักให้ลูกกินผักไม่ร...,149.0,ภูเก็ต,25
2,Klomkloam ซอสสูตรผัดกระเทียม (กลมกล่อม) ซอสปรุ...,220.0,อุทัยธานี,724
3,ผงกล้วยดิบ ผงกล้วยน้ำว้าดิบ 500g ออร์แกนิค100%...,119.0,กรุงเทพมหานคร,4800
4,ส่งฟรี‼️ผงปรุงรส ซอสปรุงรส และซอสเทอริยากิ ปรา...,240.0,กรุงเทพมหานคร,11300
...,...,...,...,...
595,เม็ดสาคูแท้ ออร์แกนิค100% จากพัทุลง ขนาด500กรัม,100.0,สงขลา,47
596,ก๋วยเตี๋ยวข้าวกล้องกึ่งสำเร็จรูปออร์แกนิค (แบบ...,39.0,ลำพูน,19
597,Tropicana Organic Cold Pressed Virgin Coconut ...,83.0,หนองบัวลำภู,34
598,กล้วยอบแห้ง​ ขายส่ง!! กล้วยอบ กล้วยเส้น ออร์แ...,51.0,เชียงใหม่,737


In [221]:
final.to_csv('shopeescrap.csv',index = False)